In [1]:
import os
import sys
import pickle

import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
import sklearn.preprocessing
from tqdm import tqdm

import models

from access_data import read_gaussian_data, read_bliss_data

2021-07-15 13:09:43.673591: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
# Path to the extracted data from Zenodo
DATA_PATH = "../galactics_attack_data/"

# Load Data and Predict $y_u$

Load BLISS signing data

In [3]:
data = read_bliss_data(DATA_PATH, "data_device_b_attack_1_2")

In [4]:
# trunkcate to first half
for col in ['yu', 'Kx', 'z']:
    data[col] = data.apply(lambda row: row[col][:512], axis=1)

# filter b to relevant bit
data['b'] = data.apply(lambda row: 1 - (row['b'] % 2), axis=1)

# Adjust Kx and yu signs
#data['Kx'] = data.apply(lambda row: row['Kx'] * np.sign(row['z']), axis=1)
#data['yu'] = data.apply(lambda row: row['yu'] * np.sign(row['z']), axis=1)
data['a'] = data.apply(lambda row: np.sign(row['z']), axis=1)

# Transpose C_matrix
data['C_matrix'] = data.apply(lambda row: row['C_matrix'].T, axis=1)
    
# compute corresponding trace index (discarding failed attempts)
data['trace_idx'] = np.cumsum(data['num_attempts']) - 1

Load $y_u$ side-channel data

In [5]:
# This path has to be the path to the traced data after preprocessing
data_path = DATA_PATH + "data_device_b_attack_1_2/"

In [6]:
yu_data = pd.read_pickle(data_path + 'yu_cw_data.pickle')

In [7]:
yu_data.reset_index(inplace=True)

Match data

In [8]:
# set of all traces that we have
TI = set(yu_data['sign_num'].unique()) & set(data['trace_idx'])

# set proper indices on BLISS and SC data
data.set_index(['trace_idx'], inplace=True)
yu_data.set_index(['sign_num', 'index'], inplace=True)

Clip data

In [9]:
yu_data['yu=0'] = yu_data['yu'] == 0
yu_data['trace'] = yu_data.apply(lambda row: row['trace'][85:110], axis=1)
trace_shape = next(iter(yu_data['trace'])).shape

Load model for $y_u$ prediction

In [10]:
yu_model = models.get_yu_model(trace_shape)

2021-07-15 13:10:44.127962: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-15 13:10:44.181294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-07-15 13:10:44.182267: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:0a:00.0 name: GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2021-07-15 13:10:44.182306: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-15 13:10:44.187541: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.

In [11]:
yu_model.load_weights('models/yu.hdf5')
yu_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                260       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                220       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________


Predict $y_u$

In [12]:
def series_to_numpy(s):
    assert len(s), 's must contain at least one row'
    l1, l2, t = len(s), len(next(iter(s))), next(iter(s)).dtype
    res = np.empty(shape=(l1, l2))
    for idx, row in enumerate(s):
        res[idx] = row
    return res

traces = series_to_numpy(yu_data['trace'])
traces = sklearn.preprocessing.StandardScaler().fit_transform(traces)
yu_data['prediction_p'] = yu_model.predict(traces, batch_size=1000)[:, 0]
yu_data['prediction'] = np.round(yu_data['prediction_p']) == 1

2021-07-15 13:10:46.543518: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 145049200 exceeds 10% of free system memory.
2021-07-15 13:10:46.644496: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-15 13:10:46.664503: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3597755000 Hz


# Attack

## Prediction Accuracy Evaluation

In [13]:
print('total number', len(yu_data))
print('false positives: ', sum(yu_data['prediction'] & ~yu_data['yu=0']))
print('false negatives: ', sum(~yu_data['prediction'] & yu_data['yu=0']))
print('accuracy', 1 - sum(yu_data['prediction'] ^ yu_data['yu=0']) / len(yu_data))

total number 1450492
false positives:  15
false negatives:  674
accuracy 0.9995249887624337


## Create Matrix $M$
We create a matricies $M_1, M_2$ using a "perfect side-channel" for control of validity, and the real side-channel using above classifier for the actual attack.

In [14]:
M1, M2 = [], []
real_sc_fp = 0
real_sc_fn = 0

for ti in tqdm(TI): # - TI_TRAIN:  # TODO do not use training data
    for i in np.where(data.loc[ti]['z1'] % 256 == 0)[0]:  # iterate over i where (zki = data.loc[ti]['z'][i] % 256) == 0
        perfect_sc_yu_is_zero = yu_data.loc[(ti, i)]['yu=0']
        real_sc_yu_is_zero = yu_data.loc[(ti, i)]['prediction']

        if perfect_sc_yu_is_zero:
            M1 += [data.loc[ti]['C_matrix'][:, i]]

        if real_sc_yu_is_zero:
            M2 += [data.loc[ti]['C_matrix'][:, i]]

        if real_sc_yu_is_zero and not perfect_sc_yu_is_zero:
            real_sc_fp += 1
        if not real_sc_yu_is_zero and perfect_sc_yu_is_zero:
            real_sc_fn += 1

M1, M2 = np.array(M1), np.array(M2)
print(f"Found matrix of rank {np.linalg.matrix_rank(M1)} via perfect side channel")
print(f"Found matrix of rank {np.linalg.matrix_rank(M2)} via real side channel")
print(f"Matrix for real SC contains {real_sc_fp} false positives.")
print(f"Matrix for real SC missed {real_sc_fn} true positives.")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1081.60it/s]


Found matrix of rank 511 via perfect side channel
Found matrix of rank 512 via real side channel
Matrix for real SC contains 1 false positives.
Matrix for real SC missed 70 true positives.


Load the correct secret key for comparison

In [15]:
real_s1 = data.iloc[0]['s1']

Do the attack for both matricies

In [16]:
import scipy as sp
from scipy import linalg

def recover_s1(M):
    ns = sp.linalg.null_space(M)
    if ns.shape[-1] == 0:
        print("No non-trivial solution found! Matrix probably noisy.")
        return np.zeros(512)
    return np.round(ns/ns.max()).reshape(512)

def key_accuracy(predicted_s1):
    return max(
        (predicted_s1 == real_s1).mean(),
        (-predicted_s1 == real_s1).mean(),
    )

print(f"Using the perfect side-channel, we recovered s1 with accuracy {key_accuracy(recover_s1(M1)):.1%}")
print(f"Using the real side-channel, we recovered s1 with accuracy {key_accuracy(recover_s1(M2)):.1%}")

Using the perfect side-channel, we recovered s1 with accuracy 100.0%
No non-trivial solution found! Matrix probably noisy.
Using the real side-channel, we recovered s1 with accuracy 69.9%


## Optimize Real Side Channel Attack
We choose a random subet of $M_2$ until we get one that yields the secret key.

In [17]:
prng = np.random.default_rng(42)
max_attempts = 50

for _ in range(max_attempts):
    subset = prng.choice(range(len(M2)), size=511, replace=False)
    M2a = M2[subset]
    if np.linalg.matrix_rank(M2a) < 511:
        print('Subset matrix had low rank.')
        continue
    accuracy = (recover_s1(M2a) == real_s1).mean()
    print(f"Got accuracy {accuracy:.1%}")
    if accuracy == 1:
        break

Got accuracy 59.4%
Got accuracy 66.6%
Got accuracy 65.0%
Got accuracy 68.4%
Got accuracy 65.8%
Got accuracy 62.1%
Got accuracy 67.6%
Got accuracy 62.3%
Got accuracy 67.0%
Got accuracy 63.1%
Got accuracy 60.9%
Got accuracy 71.1%
Got accuracy 66.2%
Got accuracy 63.5%
Got accuracy 66.2%
Got accuracy 66.6%
Got accuracy 60.9%
Got accuracy 68.4%
Got accuracy 68.6%
Got accuracy 68.4%
Got accuracy 60.2%
Got accuracy 63.5%
Got accuracy 63.3%
Got accuracy 65.4%
Got accuracy 70.1%
Got accuracy 62.3%
Got accuracy 59.8%
Got accuracy 69.5%
Got accuracy 100.0%
